# Import

In [1]:
from keras.models import load_model
from keras.models import Model
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import tensorflow as tf
from sklearn.metrics import mean_squared_error
import keras
import yfinance as yf
from matplotlib.backends.backend_pdf import PdfPages
import pickle
import json
from scipy.optimize import minimize
from datetime import datetime, timedelta

# add dividend

In [2]:
excel_file_path_div = 'settings/Dividend2018.xlsx'
div = pd.read_excel(excel_file_path_div)
print(div)

# Set symbol column as inde
div.set_index('Share', inplace=True)

      Share  Dividend
0    ALV.DE      9.00
1    DBK.DE      0.11
2   VOW3.DE      4.86
3    BMW.DE      3.50
4    ADS.DE      3.35
5    BEI.DE      0.70
6    DTE.SG      0.70
7    SAP.DE      1.50
8   1COV.DE      2.40
9    BAS.DE      3.20
10  EOAN.DE      0.43
11   RWE.DE      0.70


# Download stock data

In [3]:
def download_stock_data(stock_symbols, start_date, end_date, output_file):
    with open(output_file, 'w') as f:
        # write column headings
        f.write('Date,Open,High,Low,Close,Volume\n')
        
        #for symbol in stock_symbols:
            # load share data for the symbol (Share name) and the given time period
        stock_data = yf.download(symbol, start=start_date, end=end_date)
        stock_data = stock_data[['Open', 'High', 'Low', 'Close', 'Volume']]  # Auswahl der gewünschten Spalten
        stock_data.to_csv(f, header=False)  # Schreiben der Daten in die Datei

In [4]:
# Enter symbols
stock_symbols = ['ALV.DE', 'DBK.DE', 'VOW3.DE', 'BMW.DE', 'ADS.DE', 'BEI.DE', 'DTE.SG', 'SAP.DE', '1COV.DE', 'BAS.DE', 'EOAN.DE', 'RWE.DE']
start_date = '2014-01-01'
end_date = '2018-12-31'

# load and store the data for every share
for symbol in stock_symbols:
    output_file = f'stock_data_{symbol}.csv'
    download_stock_data(symbol, start_date, end_date, output_file)

    # read CSV and select select the desired columns
    df = pd.read_csv(output_file, usecols=['Date', 'Open', 'High', 'Low', 'Close', 'Volume'])
    
    # Search CSV file for empty lines and remove them
    with open(output_file, 'r') as file:
        lines = file.readlines()

    # filter the empty lines
    lines = [line.strip() for line in lines if line.strip()]

    # Overwrite the file with the adjusted rows
    with open(output_file, 'w') as file:
        file.write('\n'.join(lines))

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [5]:
# store the data frames for different symbols in the dictionary
dfPre = {}

for symbol in stock_symbols:
    # load CSV
    dfPre[symbol] = pd.read_csv(f'stock_data_{symbol}.csv')

    # Drop empty rows
    dfPre[symbol].dropna(inplace=True)

    # transform date to index
    dfPre[symbol].set_index('Date', inplace=True)

    # Check the number of rows
    num_rows = dfPre[symbol].shape[0]

In [6]:
# Check missing values
missing_values_beforePre = {symbol: dfPre[symbol].isnull().values.any() for symbol in stock_symbols}

# missing values are replaced with teh mean of the row before and after the actual row
for symbol in stock_symbols:
    for column in dfPre[symbol].columns:
        missing_valuesPre = dfPre[symbol][column].isnull()
        dfPre[symbol].loc[missing_valuesPre, column] = (dfPre[symbol][column].shift() + dfPre[symbol][column].shift(-1)) / 2

# Check if there are still any missing values 
missing_values_afterPre = {symbol: dfPre[symbol].isnull().values.any() for symbol in stock_symbols}

# Output of the missing values before and after the Treatment
for symbol in stock_symbols:
    print(f"Fehlende Werte vor der Behandlung für {symbol} gefunden:", missing_values_beforePre[symbol])
    print(f"Fehlende Werte nach der Behandlung für {symbol} gefunden:", missing_values_afterPre[symbol])

# Output of the length of the data frame for all symbols
for symbol in stock_symbols:
    print(f"Länge des Datensatzes für {symbol}:", len(dfPre[symbol]))

Fehlende Werte vor der Behandlung für ALV.DE gefunden: False
Fehlende Werte nach der Behandlung für ALV.DE gefunden: False
Fehlende Werte vor der Behandlung für DBK.DE gefunden: False
Fehlende Werte nach der Behandlung für DBK.DE gefunden: False
Fehlende Werte vor der Behandlung für VOW3.DE gefunden: False
Fehlende Werte nach der Behandlung für VOW3.DE gefunden: False
Fehlende Werte vor der Behandlung für BMW.DE gefunden: False
Fehlende Werte nach der Behandlung für BMW.DE gefunden: False
Fehlende Werte vor der Behandlung für ADS.DE gefunden: False
Fehlende Werte nach der Behandlung für ADS.DE gefunden: False
Fehlende Werte vor der Behandlung für BEI.DE gefunden: False
Fehlende Werte nach der Behandlung für BEI.DE gefunden: False
Fehlende Werte vor der Behandlung für DTE.SG gefunden: False
Fehlende Werte nach der Behandlung für DTE.SG gefunden: False
Fehlende Werte vor der Behandlung für SAP.DE gefunden: False
Fehlende Werte nach der Behandlung für SAP.DE gefunden: False
Fehlende Werte

# Read close values at the begin of the time period

In [7]:
# Definition of the time period
# convert in to datetime objekt to substract days for the last day before this period
start_date_obj = datetime.strptime(end_date, '%Y-%m-%d')
end_date_obj = datetime.strptime(end_date, '%Y-%m-%d')
# Substract 5 days for the new start_date and 1 day for the new end_date
# In the next block the last date will be extracted 
new_start_date = start_date_obj - timedelta(days=5)
new_end_date = end_date_obj

print(new_start_date)
print(new_end_date)

# Load and store the data for every symbol (share)
for symbol in stock_symbols:
    output_file = f'stock_data_begin{symbol}.csv'
    download_stock_data(symbol, new_start_date, new_end_date, output_file)

    # Search and delete empty rows in the CSV file
    with open(output_file, 'r') as file:
        lines = file.readlines()

    # Filter the empty rows
    lines = [line.strip() for line in lines if line.strip()]

    # Overwrite the file with the cleaned data
    with open(output_file, 'w') as file:
        file.write('\n'.join(lines))


[*********************100%%**********************]  1 of 1 completed

2018-12-26 00:00:00
2018-12-31 00:00:00



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [8]:
# create data frame for the collected data 
combined_data_begin = pd.DataFrame(columns=['Symbol', 'Close_Value'])

# load and store the data for every symbol (share)
for symbol in stock_symbols:
    output_file = f'stock_data_begin{symbol}.csv'
    # read CSV file 
    df = pd.read_csv(output_file)

    # Select the last row only
    last_row = df.tail(1)

     # Write symbol and close value to the data frame
    data = {'Symbol': symbol, 'Close_Value': last_row['Close'].iloc[0]}
    combined_data_begin = pd.concat([combined_data_begin, pd.DataFrame(data, index=[0])], ignore_index=True)
    
# Add a row for the risk-free interest rate with the specified value
risk_free_rate = 0.3 # Input in percentage
risk_free_rate = risk_free_rate # Risk free rate per month
risk_free_data = {'Symbol': 'Risk_Free', 'Close_Value': 100}
combined_data_begin = combined_data_begin.append(risk_free_data, ignore_index=True)

print(combined_data_begin)

       Symbol  Close_Value
0      ALV.DE   175.139999
1      DBK.DE     6.967000
2     VOW3.DE   138.919998
3      BMW.DE    70.699997
4      ADS.DE   182.399994
5      BEI.DE    91.160004
6      DTE.SG    13.690000
7      SAP.DE    86.930000
8     1COV.DE    43.180000
9      BAS.DE    60.400002
10    EOAN.DE     8.627000
11     RWE.DE    18.965000
12  Risk_Free   100.000000


C:\Users\Samuel\AppData\Local\Temp\ipykernel_15872\1559878281.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_data_begin = combined_data_begin.append(risk_free_data, ignore_index=True)


# load share value from 2019

In [9]:
start_date = '2019-12-01'
end_date = '2019-12-31'
print(f'Startdatum: {start_date}, Enddatum: {end_date}')
# Load and store the symbols for every symbol (share)
for symbol in stock_symbols:
    output_file = f'stock_data_{symbol}_2019.csv'
    download_stock_data(symbol, start_date, end_date, output_file)

    # search and delete for empty rows in the CSV 
    with open(output_file, 'r') as file:
        lines = file.readlines()

    # Filter the empty rows
    lines = [line.strip() for line in lines if line.strip()]

    # Overwrite the file with the cleaned rows
    with open(output_file, 'w') as file:
        file.write('\n'.join(lines))


[*********************100%%**********************]  1 of 1 completed


Startdatum: 2019-12-01, Enddatum: 2019-12-31


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [10]:
# list for the las close value of the shares
last_close_values = []

# load for each symbol
for symbol in stock_symbols:
    # file name of the csv data
    csv_file = f'stock_data_{symbol}_2019.csv'
    
    # load CSV data
    df = pd.read_csv(csv_file)
    
    # set date as index
    df.set_index('Date', inplace=True)
    
    # collect just the last close value and write it to the list
    last_close_value = df['Close'].iloc[-1]
    last_close_values.append((symbol, last_close_value))
print(last_close_values[0][1])

# Add a row for the risk-free interest rate with a value of 1
last_close_values.append(('Risk_Free', 100 + risk_free_rate))

# print the list
print("Liste der letzten Close-Werte:")
for symbol, last_close_value in last_close_values:
    print(f"{symbol}: {last_close_value}")

218.3999938964844
Liste der letzten Close-Werte:
ALV.DE: 218.3999938964844
DBK.DE: 6.916999816894531
VOW3.DE: 176.24000549316406
BMW.DE: 73.13999938964844
ADS.DE: 289.79998779296875
BEI.DE: 106.6500015258789
DTE.SG: 13.6899995803833
SAP.DE: 120.31999969482422
1COV.DE: 41.45000076293945
BAS.DE: 67.3499984741211
EOAN.DE: 9.52400016784668
RWE.DE: 27.350000381469727
Risk_Free: 100.3


# Split into blocks of 252 days (252 business days per year)

In [11]:
# Dictionary for the splitted data
df_split = {}

# Quantity of the blocks per symbol
block_size = 252

for symbol, df in dfPre.items():
    # Quantity of the rows in the data frame
    num_rows = len(df)

    # Split the data frame in to blocks
    split_dfs = np.array_split(df, num_rows // block_size)

    # Add the solited blocks to the dictionary
    df_split[symbol] = split_dfs

# Output result
for symbol, split_dfs in df_split.items():
    print(f"Symbol: {symbol}")
    for i, df_block in enumerate(split_dfs):
        print(f"Block {i+1}:")
        print(df_block)
        print()

Symbol: ALV.DE
Block 1:
                  Open        High         Low       Close   Volume
Date                                                               
2014-01-02  130.949997  131.399994  127.849998  128.149994  1524683
2014-01-03  128.000000  128.449997  127.150002  127.900002   853713
2014-01-06  127.650002  128.800003  127.599998  128.399994   900878
2014-01-07  128.350006  130.350006  128.149994  130.100006  1189652
2014-01-08  130.149994  130.149994  128.850006  129.399994  1214204
...                ...         ...         ...         ...      ...
2014-12-23  139.500000  140.000000  138.899994  139.699997  1040720
2014-12-29  139.000000  139.399994  137.050003  139.250000  1083568
2014-12-30  138.149994  138.300003  137.350006  137.350006   695403
2015-01-02  137.300003  137.850006  135.750000  136.699997  1494235
2015-01-05  136.500000  137.500000  133.149994  133.350006  2091329

[254 rows x 5 columns]

Block 2:
                  Open        High         Low       Close

# Calculate the return for one year

In [12]:
# Dictionary for the increase/decrease per symbol
change_per_symbol = {}

for symbol, split_dfs in df_split.items():
    changes = []

    # Iterate over every block of the current symbol
    for df_block in split_dfs:
        # Extract the first and the lat close value of the current block
        first_close = df_block.iloc[0]['Close']
        last_close = df_block.iloc[-1]['Close']

        # Calculate the change of the close value
        change = (last_close - first_close + div.loc[symbol, 'Dividend']) / first_close * 100  # in Prozent
        print(symbol)
        print(div.loc[symbol, 'Dividend'])
        changes.append(change)

    # Save the Change for the current symbol
    change_per_symbol[symbol] = changes

# Output result
for symbol, changes in change_per_symbol.items():
    print(f"Symbol: {symbol}")
    for i, change in enumerate(changes):
        print(f"Block {i+1}: {change:.2f}%")
    print()


ALV.DE
9.0
ALV.DE
9.0
ALV.DE
9.0
ALV.DE
9.0
ALV.DE
9.0
DBK.DE
0.11
DBK.DE
0.11
DBK.DE
0.11
DBK.DE
0.11
DBK.DE
0.11
VOW3.DE
4.86
VOW3.DE
4.86
VOW3.DE
4.86
VOW3.DE
4.86
VOW3.DE
4.86
BMW.DE
3.5
BMW.DE
3.5
BMW.DE
3.5
BMW.DE
3.5
BMW.DE
3.5
ADS.DE
3.35
ADS.DE
3.35
ADS.DE
3.35
ADS.DE
3.35
ADS.DE
3.35
BEI.DE
0.7
BEI.DE
0.7
BEI.DE
0.7
BEI.DE
0.7
BEI.DE
0.7
DTE.SG
0.7
DTE.SG
0.7
DTE.SG
0.7
DTE.SG
0.7
DTE.SG
0.7
SAP.DE
1.5
SAP.DE
1.5
SAP.DE
1.5
SAP.DE
1.5
SAP.DE
1.5
1COV.DE
2.4
1COV.DE
2.4
1COV.DE
2.4
BAS.DE
3.2
BAS.DE
3.2
BAS.DE
3.2
BAS.DE
3.2
BAS.DE
3.2
EOAN.DE
0.43
EOAN.DE
0.43
EOAN.DE
0.43
EOAN.DE
0.43
EOAN.DE
0.43
RWE.DE
0.7
RWE.DE
0.7
RWE.DE
0.7
RWE.DE
0.7
RWE.DE
0.7
Symbol: ALV.DE
Block 1: 11.08%
Block 2: 23.97%
Block 3: 5.92%
Block 4: 26.76%
Block 5: -3.84%

Symbol: DBK.DE
Block 1: -25.12%
Block 2: -8.35%
Block 3: -18.26%
Block 4: 1.51%
Block 5: -55.42%

Symbol: VOW3.DE
Block 1: -9.65%
Block 2: -28.83%
Block 3: 19.94%
Block 4: 22.72%
Block 5: -13.62%

Symbol: BMW.DE
Block 1: 6.03%
Block 2

# Calculate the mean return over the period of 4 years

In [13]:
# Dictionary for the mean return per symbol
average_return_per_symbol = {}

for symbol, changes in change_per_symbol.items():
    # Calculate the mean return for the current symbol
    average_return = sum(changes) / len(changes)
    average_return_per_symbol[symbol] = average_return

# Add Risk_Free symbol with risk_free_rate average return
average_return_per_symbol['Risk_Free'] = risk_free_rate    

# Output result
for symbol, avg_return in average_return_per_symbol.items():
    print(f"Durchschnittliche Rendite für {symbol}: {avg_return:.2f}%")
    
# Convert mean return to DataFrame
average_return_per_symbol_df = pd.DataFrame(list(average_return_per_symbol.items()), columns=['Symbol', 'Average_Return'])
print(average_return_per_symbol_df)


Durchschnittliche Rendite für ALV.DE: 12.78%
Durchschnittliche Rendite für DBK.DE: -21.13%
Durchschnittliche Rendite für VOW3.DE: -1.89%
Durchschnittliche Rendite für BMW.DE: 1.53%
Durchschnittliche Rendite für ADS.DE: 26.99%
Durchschnittliche Rendite für BEI.DE: 6.50%
Durchschnittliche Rendite für DTE.SG: 8.96%
Durchschnittliche Rendite für SAP.DE: 10.79%
Durchschnittliche Rendite für 1COV.DE: 42.27%
Durchschnittliche Rendite für BAS.DE: 2.99%
Durchschnittliche Rendite für EOAN.DE: 1.82%
Durchschnittliche Rendite für RWE.DE: 4.81%
Durchschnittliche Rendite für Risk_Free: 0.30%
       Symbol  Average_Return
0      ALV.DE       12.778761
1      DBK.DE      -21.126335
2     VOW3.DE       -1.886427
3      BMW.DE        1.525783
4      ADS.DE       26.993529
5      BEI.DE        6.500040
6      DTE.SG        8.964563
7      SAP.DE       10.791699
8     1COV.DE       42.274826
9      BAS.DE        2.986745
10    EOAN.DE        1.817033
11     RWE.DE        4.806770
12  Risk_Free        0.30

In [14]:
# Dictionary for the standard deviation per symbol
standard_deviation_per_symbol = {}

for symbol, changes in change_per_symbol.items():
    
    # Calculate the difference to the power of 2
    squared_diff = [(change - average_return_per_symbol[symbol]) ** 2 for change in changes]
    
    # Sum up the differences
    sum_squared_diff = sum(squared_diff)

    # Calculate the standard deviation
    standard_deviation = np.sqrt(sum_squared_diff / (len(changes) - 1))
    
    # Save the standard deviation in the dictionary
    standard_deviation_per_symbol[symbol] = standard_deviation

# Add Risk_Free symbol with risk_free_rate average return
standard_deviation_per_symbol['Risk_Free'] = 0.0  

# Output result
for symbol, std_dev in standard_deviation_per_symbol.items():
    print(f"Standardabweichung für {symbol}: {std_dev:.2f}")
    
# Convert mean return to DataFrame
standard_deviation_per_symbol_df = pd.DataFrame(list(standard_deviation_per_symbol.items()), columns=['Symbol', 'Standard_Deviation'])
print(standard_deviation_per_symbol_df)

Standardabweichung für ALV.DE: 12.72
Standardabweichung für DBK.DE: 21.66
Standardabweichung für VOW3.DE: 22.39
Standardabweichung für BMW.DE: 9.83
Standardabweichung für ADS.DE: 46.56
Standardabweichung für BEI.DE: 15.61
Standardabweichung für DTE.SG: 14.92
Standardabweichung für SAP.DE: 16.50
Standardabweichung für 1COV.DE: 81.95
Standardabweichung für BAS.DE: 25.57
Standardabweichung für EOAN.DE: 26.51
Standardabweichung für RWE.DE: 36.02
Standardabweichung für Risk_Free: 0.00
       Symbol  Standard_Deviation
0      ALV.DE           12.717785
1      DBK.DE           21.656503
2     VOW3.DE           22.392023
3      BMW.DE            9.831779
4      ADS.DE           46.558759
5      BEI.DE           15.605312
6      DTE.SG           14.915638
7      SAP.DE           16.502918
8     1COV.DE           81.947789
9      BAS.DE           25.567918
10    EOAN.DE           26.514237
11     RWE.DE           36.018191
12  Risk_Free            0.000000


# Historic data from 2018 for the covarianz matrix

In [15]:
# Definition of the time period
start_date2018 = '2018-01-01'
end_date2018 = '2018-12-31'
print(end_date2018)
# Load and store the data for every symbol (share)
for symbol in stock_symbols:
    output_file = f'stock_data_{symbol}_2018.csv'
    download_stock_data(symbol, start_date2018, end_date2018, output_file)

    # Search and delete empty rows in the CSV file
    with open(output_file, 'r') as file:
        lines = file.readlines()

    # Filter the empty rows
    lines = [line.strip() for line in lines if line.strip()]

    # Overwrite the file with the cleaned rows
    with open(output_file, 'w') as file:
        file.write('\n'.join(lines))


2018-12-31


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


# Combine historical data to one file

In [16]:
# Data frame for the combined data
combined_historical2018_data = pd.DataFrame()

# load and combine the data for each share
for symbol in stock_symbols:
    # file name for the csv file
    csv_file = f'stock_data_{symbol}_2018.csv'
    
    # read CSV file and set date to index
    df = pd.read_csv(csv_file, index_col='Date', parse_dates=True)
    
    # change name of the close column
    df.rename(columns={'Close': symbol}, inplace=True)
    
    # write data to the combined file
    combined_historical2018_data = pd.concat([combined_historical2018_data, df[symbol]], axis=1)

# drop rows with NaN values
combined_historical2018_data.dropna(inplace=True)

# Initialize the initial and final values for "Risk_Free"
risk_free_initial = 100
risk_free_final = 100 + risk_free_rate

# Create a series for "Risk_Free" with increasing values
risk_free_values = np.linspace(risk_free_initial, risk_free_final, len(combined_historical2018_data))

# Add the "Risk_Free" series to the DataFrame
combined_historical2018_data['Risk_Free'] = risk_free_values

# create a CSV file for the data
combined_historical2018_data.to_csv('combined_historical2018_stock_data.csv')

print(combined_historical2018_data)

                ALV.DE     DBK.DE     VOW3.DE     BMW.DE      ADS.DE  \
2018-01-02  192.699997  15.958000  165.699997  86.400002  167.149994   
2018-01-03  193.000000  15.910000  171.440002  86.860001  168.050003   
2018-01-04  195.500000  16.332001  174.440002  87.480003  170.250000   
2018-01-05  198.559998  15.490000  179.199997  88.500000  172.050003   
2018-01-08  198.860001  15.340000  179.839996  89.669998  172.750000   
...                ...        ...         ...        ...         ...   
2018-12-19  175.940002   7.530000  146.880005  73.449997  187.050003   
2018-12-20  174.960007   7.000000  143.899994  71.839996  183.750000   
2018-12-21  175.020004   7.042000  143.300003  71.930000  184.750000   
2018-12-27  172.160004   6.750000  137.440002  69.860001  180.100006   
2018-12-28  175.139999   6.967000  138.919998  70.699997  182.399994   

                BEI.DE  DTE.SG     SAP.DE    1COV.DE     BAS.DE  EOAN.DE  \
2018-01-02   96.199997  14.872  92.800003  85.639999  91.30

# Mean-Variance Optimization

In [17]:
def mean_variance_optimization(expected_returns, standard_deviations, covariance_matrix):
    n = len(expected_returns)
    initial_weights = np.array([1/n] * n)  # Start value of the weighs
    bounds = [(0, 1)] * n  # Border of the weighs (0-100% for every symbol)

    # Minimize the negative sharpe ratio
    def negative_sharpe(weights, expected_returns, standard_deviations, covariance_matrix):
        portfolio_return = np.dot(weights, expected_returns)
        portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(covariance_matrix, weights)))
        return -portfolio_return / portfolio_volatility

    result = minimize(negative_sharpe, initial_weights, args=(expected_returns, standard_deviations, covariance_matrix), bounds=bounds, constraints={'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    return result.x

In [18]:
def allocate_portfolio(historical_data, initial_capital, expected_returns_df, standard_deviations_df):
    # Calculate expected returns based on the predictions
    expected_returns = expected_returns_df['Average_Return'].values
    
    # Calculate standard deviations based on the predictions
    standard_deviations = standard_deviations_df['Standard_Deviation'].values
    
    # Calculate the covariance matrix of the returns
    covariance_matrix = historical_data.cov()
           
    print(covariance_matrix)
    
    # Perform mean-variance optimization to obtain optimal weights
    optimal_weights = mean_variance_optimization(expected_returns, standard_deviations, covariance_matrix)

    # Calculate the allocation of assets based on the optimal weights and the available capital
    asset_allocation = initial_capital * optimal_weights

    return asset_allocation

In [19]:
initial_capital = 1000 # 1000€ start capital
portfolio_allocation = allocate_portfolio(combined_historical2018_data, initial_capital, average_return_per_symbol_df, standard_deviation_per_symbol_df)
print(portfolio_allocation)

              ALV.DE     DBK.DE     VOW3.DE     BMW.DE      ADS.DE     BEI.DE  \
ALV.DE     55.719135  11.832723   69.633719  29.543884  -14.913587   0.700969   
DBK.DE     11.832723   4.459489   21.177797  10.619505  -14.403096  -0.613004   
VOW3.DE    69.633719  21.177797  173.761508  68.923353  -60.940672  -4.044547   
BMW.DE     29.543884  10.619505   68.923353  39.954812  -18.970997   3.151594   
ADS.DE    -14.913587 -14.403096  -60.940672 -18.970997  151.098444   7.531624   
BEI.DE      0.700969  -0.613004   -4.044547   3.151594    7.531624  13.783059   
DTE.SG      1.824277   0.309639    1.939910   0.824193   -0.172976   0.808876   
SAP.DE     -7.402701  -4.671514  -35.207519  -7.149319   32.796906  19.146100   
1COV.DE    52.288467  23.954566  105.094400  71.570572  -67.992106   6.309557   
BAS.DE     38.744048  16.118842   87.074221  52.709029  -45.541645   8.408377   
EOAN.DE    -0.464208  -0.189888   -0.824805   0.054390    0.739193   0.973982   
RWE.DE     -3.476472  -1.526

In [20]:
# Initialisation of the portfolio
portfolio_value = 0
# initialize the list for the quantity of the shares, which have to be purchased
shares_to_buy_list = []
# Purchase the shares based on the allocations
for i, allocation in enumerate(portfolio_allocation):
    # Calculate the quantity of the shares, which should be purchased with this allocation
    shares_to_buy = (allocation) / combined_data_begin['Close_Value'][i]
    shares_to_buy_list.append(shares_to_buy)
    print(shares_to_buy)

    # Calculate the value of the stock of the purchased shares
    value_of_stock = shares_to_buy * combined_data_begin['Close_Value'][i]
    print(value_of_stock)
    
    # Add the value of the purchased share to the portfolio
    portfolio_value += value_of_stock
    print(portfolio_value)
    print("-----------------------------")

# Output of the total value of the portfolio at the end of the period to be predicted
print("Gesamtwert des Portfolios am Ende des ersten Monats von 2019:", portfolio_value)
print(shares_to_buy_list)

1.1617419926067725e-07
2.0346749187607908e-05
2.0346749187607908e-05
-----------------------------
1.9429451427834844e-12
1.353649882459603e-11
2.0346762724106734e-05
-----------------------------
0.003367274921447624
0.46778182592183937
0.4678021726845635
-----------------------------
1.0439841893017589e-12
7.380967899764745e-11
0.46780217275837316
-----------------------------
0.0
0.0
0.46780217275837316
-----------------------------
2.15226614334514e-07
1.96200589509177e-05
0.46782179281732406
-----------------------------
6.465392667261758
88.51122290182674
88.97904469464406
-----------------------------
4.546302300782913e-07
3.9521006039448e-05
88.9790842156501
-----------------------------
0.1203069258223303
5.194853093722982
94.17393730937309
-----------------------------
4.9783939578877346e-08
3.006950026528455e-06
94.17394031632311
-----------------------------
2.4532403693215245e-11
2.1164104310518756e-10
94.17394031653475
-----------------------------
6.1429574921683465e-12


In [21]:
# Define portfolio value
new_portfolio_value = 0
# Loop over the indices of the two lists
for i in range(len(shares_to_buy_list)):
    # convert to float
    last_close_value = float(last_close_values[i][1])
    print(last_close_value)
    print(shares_to_buy_list[i])
    #Calculate the new value of the share and add the value to the list
    new_portfolio_value += (last_close_value * shares_to_buy_list[i])
    print(new_portfolio_value)
    print("----------------------------")

print("Wert des Portfolios:")
print(new_portfolio_value)

218.3999938964844
1.1617419926067725e-07
2.5372444409460875e-05
----------------------------
6.916999816894531
1.9429451427834844e-12
2.5372457848812073e-05
----------------------------
176.24000549316406
0.003367274921447624
0.5934739231107716
----------------------------
73.13999938964844
1.0439841893017589e-12
0.5934739231871287
----------------------------
289.79998779296875
0.0
0.5934739231871287
----------------------------
106.6500015258789
2.15226614334514e-07
0.5934968771058758
----------------------------
13.6899995803833
6.465392667261758
89.10471977893262
----------------------------
120.31999969482422
4.546302300782913e-07
89.10477448004177
----------------------------
41.45000076293945
0.1203069258223303
94.09149664716426
----------------------------
67.3499984741211
4.9783939578877346e-08
94.09150000011252
----------------------------
9.52400016784668
2.4532403693215245e-11
94.09150000034616
----------------------------
27.350000381469727
6.1429574921683465e-12
94.091500